[itsandrewgao](https://twitter.com/itsandrewgao) my very rushed and shitty attempt at implementing just the mLSTM to predict sine waves

In [2]:
from google.colab import files
uploaded = files.upload()

Saving October-13-2023-19-20-46.csv to October-13-2023-19-20-46.csv


<a target="_blank" href="https://colab.research.google.com/github/andrewgcodes/xlstm/blob/main/mLSTM.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [5]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

df = pd.read_csv('your_csv_file.csv')
moisture_pot1 = df['Moisture Pot 1'].values
moisture_pot2 = df['Moisture Pot 2'].values

moisture_pot1_tensor = torch.tensor(moisture_pot1, dtype=torch.float32).unsqueeze(0)
moisture_pot2_tensor = torch.tensor(moisture_pot2, dtype=torch.float32).unsqueeze(0)

input_data = torch.cat([moisture_pot1_tensor, moisture_pot2_tensor], dim=0)

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, mem_dim):
        super(mLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.mem_dim = mem_dim
        self.Wq = nn.Parameter(torch.randn(hidden_size, input_size))
        self.bq = nn.Parameter(torch.randn(hidden_size, 1))
        self.Wk = nn.Parameter(torch.randn(mem_dim, input_size))
        self.bk = nn.Parameter(torch.randn(mem_dim, 1))
        self.Wv = nn.Parameter(torch.randn(mem_dim, input_size))
        self.bv = nn.Parameter(torch.randn(mem_dim, 1))
        self.wi = nn.Parameter(torch.randn(1, input_size))
        self.bi = nn.Parameter(torch.randn(1))
        self.wf = nn.Parameter(torch.randn(1, input_size))
        self.bf = nn.Parameter(torch.randn(1))
        self.Wo = nn.Parameter(torch.randn(hidden_size, input_size))
        self.bo = nn.Parameter(torch.randn(hidden_size, 1))
        self.reset_parameters()

    def reset_parameters(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                nn.init.xavier_uniform_(p.data)
            else:
                nn.init.zeros_(p.data)

    def forward(self, x, states):
        (C_prev, n_prev) = states
        qt = torch.matmul(self.Wq, x) + self.bq
        kt = (1 / math.sqrt(self.mem_dim)) * (torch.matmul(self.Wk, x) + self.bk)
        vt = torch.matmul(self.Wv, x) + self.bv

        x = x.view(x.size(0), -1)
        for t in range(x.size(1)):
        input_t = x[:, t, :]
        ht, states = self._mlstm_cell(input_t, states)

        it = torch.exp(torch.matmul(self.wi, x) + self.bi)
        ft = torch.sigmoid(torch.matmul(self.wf, x) + self.bf)

        vt = vt.squeeze()
        kt = kt.squeeze()

        C = ft * C_prev + it * torch.ger(vt, kt)
        n = ft * n_prev + it * kt.unsqueeze(1)

        max_nqt = torch.max(torch.abs(torch.matmul(n.T, qt)), torch.tensor(1.0))
        h_tilde = torch.matmul(C, qt) / max_nqt
        ot = torch.sigmoid(torch.matmul(self.Wo, x) + self.bo)
        ht = ot * h_tilde

        return ht, (C, n)

    def init_hidden(self):
        return (torch.zeros(self.mem_dim, self.mem_dim),
                torch.zeros(self.mem_dim, 1))


input_size = 2
hidden_size = 10
mem_dim = 10

model = mLSTM(input_size, hidden_size, mem_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# 训练
num_epochs = 200
seq_len = input_data.shape[1]

for epoch in range(num_epochs):
    states = model.init_hidden()
    optimizer.zero_grad()
    total_loss = 0

    for t in range(seq_len - 1):
        input_t = input_data[:, t].unsqueeze(1)
        target_t = input_data[:, t + 1].unsqueeze(1)
        y_pred, states = model(input_t, states)
        loss = criterion(y_pred, target_t)
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss.item()}')

# result
test_output = []
states = model.init_hidden()
for t in range(seq_len - 1):
    x = data[:, t]
    y_pred, states = model(x, states)
    test_output.append(y_pred.detach().numpy().ravel()[0])

plt.figure(figsize=(10, 4))
plt.title('Original vs. Predicted Moisture')
plt.plot(data.squeeze().numpy(), label='Original')
plt.plot(test_output, label='Predicted')
plt.legend()
plt.show()


KeyboardInterrupt: 